<a href="https://colab.research.google.com/github/RodrigoEduard/TFMviu/blob/main/RTraMineR_Norte_Grande.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
library(readxl)
conduccion_v1 <- read_excel("conduccion_220723_sinVacios.xlsx")

In [2]:
conduccion_v1$`MES INGRESO` <- as.numeric(conduccion_v1$`MES INGRESO`)
# Crear una nueva columna 'FECHA' combinando 'AÑO INGRESO' y 'MES INGRESO' en el formato "YYYY-MM"
conduccion_v1$FECHA <- paste('a', substr(conduccion_v1$`AÑO INGRESO`, 3, 4), '.', sprintf("%02d", conduccion_v1$`MES INGRESO`), sep = "")

In [3]:
#ZONA: NORTE GRANDE ----
regiones <- c('DE ARICA Y PARINACOTA', 'DE TARAPACA', 'DE ANTOFAGASTA')
dat <- subset(conduccion_v1, `NOMBRE REGION` %in% regiones)
dat = table(dat$`NOMBRE COMUNA`,dat$`FECHA`)
pde_f = 100*(dat/rowSums(dat))
pde_f = as.data.frame(pde_f)
max = max(pde_f[,3],na.rm=T)
max = round(max,digits = 0)
mediana = median(pde_f[,3],na.rm=T)  #calcula la mediana de la tercera columna del dataframe pde_f, ignorando cualquier valor NA, y luego asigna ese valor a la variable mediana.
pde_f$clase = 0
pde_f$clase[pde_f$Freq == mediana] <- 'IGUAL'
pde_f$clase[pde_f$Freq > mediana] <- 'SOBRE'
pde_f$clase[pde_f$Freq < mediana] <- 'BAJO'
pde_f$clase[pde_f$Freq == 0] <- 'SIN_REG'

In [ ]:
# Plot
library(ggplot2)
library(magrittr)
pde_f %>%
  ggplot( aes(x=Var2, y=Freq, group=Var1)) +
  geom_line()+facet_wrap(vars(Var1), ncol = 3,scales = "free")+theme(legend.position = "none") + ylim(0, max+5) +
  ggtitle("TODAS LAS COMUNAS") + theme(plot.title = element_text(size = 8))

In [ ]:
library(tidyverse)
TRAN <- pde_f %>%
  rename(id = Var1) %>%
  group_by(id, Var2) %>%
  summarise(clase = first(clase), .groups = "drop") %>%
  pivot_wider(names_from = Var2, values_from = clase)

TRAN <- as.data.frame(TRAN)